## Convert the model
Convert the model to onnx so that we can run this with C++ onnx runtime.

In [2]:
import torch
import torchvision.models as models
import os
import torch.nn as nn

BASE_DIR = r"D:\braineye"
MODEL_DIR = os.path.join(BASE_DIR, "models")

def load_model(model_path):
    model = models.mobilenet_v2(weights='DEFAULT')
    model = model.to('cpu')
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 1)
    state_dict = torch.load(model_path, weights_only=True, map_location=torch.device('cpu')) 
    model.load_state_dict(state_dict)
    return model


# 1. Load your trained model
model_path = os.path.join(MODEL_DIR, "age_prediction_0.0.1.pt")  # path to your model
model = load_model(model_path)

# 2. If the model was saved using model.state_dict()
# Uncomment and use the same architecture as during training
# model = models.mobilenet_v2()  # or mobilenet_v3_small(), etc.
# model.load_state_dict(torch.load(model_path, map_location='cpu'))

# 3. Set model to evaluation mode
model.eval()

# 4. Create a dummy input matching the model’s expected input size
# Example for 128x128 RGB image
dummy_input = torch.randn(1, 3, 224, 224, device='cpu')

# 5. Define output ONNX file name
onnx_file = os.path.join(MODEL_DIR, "age_prediction_0.0.1.onnx")

# 6. Export to ONNX
torch.onnx.export(
    model,                     # model being run
    dummy_input,               # model input (or a tuple for multiple inputs)
    onnx_file,                 # where to save the model
    export_params=True,        # store the trained parameter weights
    opset_version=17,          # ONNX version (17 is latest and recommended)
    do_constant_folding=True,  # optimize constant expressions
    input_names=['input'],     # input tensor name
    output_names=['output'],   # output tensor name
    dynamic_axes={             # allow dynamic batch size
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print(f"✅ Model successfully converted to {onnx_file}")


C:\Users\d.shrestha\AppData\Local\Temp\ipykernel_3260\3502888377.py:39: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


✅ Model successfully converted to D:\braineye\models\age_prediction_0.0.1.onnx
